# Notebook 1: Persona Data Generation

Generate contrastive response pairs for each persona using the Deep Infra API.

For each persona × each evaluation prompt, we generate:
- A **trait response** (system prompt exhibiting the persona)
- An **anti-trait response** (system prompt exhibiting the opposite)

These contrastive pairs are what we'll feed through GPT-2 medium in Notebook 2 to extract persona vectors.

Progress saves incrementally to `data/persona_responses.json` so you can stop and resume without re-spending on API calls.

In [1]:
import sys
sys.path.insert(0, '..')

from src.personas import PERSONAS, EVALUATION_PROMPTS
from src.api_client import get_client, generate_contrastive_pairs

## Inspect personas and prompts

In [2]:
print(f"Number of personas: {len(PERSONAS)}")
print(f"Number of evaluation prompts: {len(EVALUATION_PROMPTS)}")
print(f"Total API calls needed: {len(PERSONAS) * len(EVALUATION_PROMPTS) * 2}")
print()
for p in PERSONAS:
    print(f"  - {p['label']}: {p['description']}")

Number of personas: 9
Number of evaluation prompts: 40
Total API calls needed: 720

  - Urgent & Direct: Agent who cuts to the chase — apologizes fast, leads with the fix, no fluff.
  - Warm & Patient: Agent who takes their time, explains step by step, makes you feel welcome.
  - Polished & Premium: Agent with a white-glove tone — proactive, personalized, goes above and beyond.
  - Technical & Precise: Agent who gives detailed, specific answers with exact steps and specs.
  - Empathetic & Compassionate: Agent who leads with feelings — acknowledges emotions before solving problems.
  - Friendly & Conversational: Agent who chats naturally — personable, uses humor, builds rapport.
  - Value-Focused: Agent who proactively surfaces deals, savings, and cost-effective options.
  - Gentle & Simple: Agent who uses very simple language, short sentences, and a kind tone — like a teacher.
  - Cautious & Thorough: Agent who covers all the caveats, edge cases, and fine print upfront.


## Test a single persona + prompt pair

Before running the full batch, let's verify the API works and see what a contrastive pair looks like.

In [3]:
from src.api_client import generate_response

client = get_client()

test_persona = PERSONAS[0]  # Frustrated Customer
test_prompt = EVALUATION_PROMPTS[0]

print(f"Persona: {test_persona['label']}")
print(f"Prompt: {test_prompt}")
print()

trait_resp = generate_response(client, test_persona['system_prompt'], test_prompt)
print("=== TRAIT RESPONSE ===")
print(trait_resp)
print()

anti_resp = generate_response(client, test_persona['anti_system_prompt'], test_prompt)
print("=== ANTI-TRAIT RESPONSE ===")
print(anti_resp)

Persona: Urgent & Direct
Prompt: My order hasn't arrived and it's been two weeks. What's going on?

=== TRAIT RESPONSE ===
Sorry for the delay. Send me your order number. I’ll locate the shipment, confirm its status, and either expedite delivery or issue a refund. Let me know which you prefer.

=== ANTI-TRAIT RESPONSE ===
Hello there! 🌟 First of all, thank you ever so much for reaching out and taking the time to let us know about the situation with your order. I truly appreciate your patience and understand how frustrating it can be when something you’re looking forward to doesn’t arrive on schedule. It’s completely normal to feel a little uneasy when a delivery takes longer than expected, especially when you’ve been waiting for a couple of weeks now. 

Let me start by saying how sorry we are for any inconvenience this has caused you. We strive to make every experience as smooth and delightful as possible, and when we fall short of that, it’s important to us to understand why and to ma

## Generate all contrastive pairs

This will take a while. Progress is saved after every pair, so it's safe to interrupt and resume.

In [4]:
client = get_client()

results = generate_contrastive_pairs(
    client=client,
    personas=PERSONAS,
    prompts=EVALUATION_PROMPTS,
    save_path="../data/persona_responses.json",
)

Progress: 10/360
Progress: 20/360
Progress: 30/360
Progress: 40/360
Progress: 50/360
Progress: 60/360
Progress: 70/360
Progress: 80/360
Progress: 90/360
Progress: 100/360
Progress: 110/360
Progress: 120/360
Progress: 130/360
Progress: 140/360
Progress: 150/360
Progress: 160/360
Progress: 170/360
Progress: 180/360
Progress: 190/360
Progress: 200/360
Progress: 210/360
Progress: 220/360
Progress: 230/360
Progress: 240/360
Progress: 250/360
Progress: 260/360
Progress: 270/360
Progress: 280/360
Progress: 290/360
Progress: 300/360
Progress: 310/360
Progress: 320/360
Progress: 330/360
Progress: 340/360
Progress: 350/360
Progress: 360/360
Done! 360/360 pairs saved to ../data/persona_responses.json


## Inspect saved data

In [7]:
import json

with open("../data/persona_responses.json") as f:
    data = json.load(f)

print(f"Personas saved: {list(data.keys())}")
for name, prompts in data.items():
    print(f"  {name}: {len(prompts)} prompt pairs")

Personas saved: ['urgent_direct', 'warm_patient', 'polished_premium', 'technical_precise', 'empathetic_compassionate', 'friendly_conversational', 'value_focused', 'gentle_simple', 'cautious_thorough']
  urgent_direct: 40 prompt pairs
  warm_patient: 40 prompt pairs
  polished_premium: 40 prompt pairs
  technical_precise: 40 prompt pairs
  empathetic_compassionate: 40 prompt pairs
  friendly_conversational: 40 prompt pairs
  value_focused: 40 prompt pairs
  gentle_simple: 40 prompt pairs
  cautious_thorough: 40 prompt pairs


In [8]:
# Look at a random sample
import random

persona = random.choice(list(data.keys()))
prompt_key = random.choice(list(data[persona].keys()))
sample = data[persona][prompt_key]

print(f"Persona: {persona}")
print(f"Prompt: {sample['prompt']}")
print()
print("=== TRAIT ===")
print(sample['trait_response'][:500])
print()
print("=== ANTI-TRAIT ===")
print(sample['anti_response'][:500])

Persona: cautious_thorough
Prompt: I'm dealing with a family emergency and need to expedite a return.

=== TRAIT ===
I’m very sorry to hear that you’re dealing with a family emergency. I’ll do my best to walk you through the return process and let you know where we can potentially speed things up, while also outlining the limits of what we can guarantee.

---

### 1. Gather the essential information first  
In order to even consider an expedited handling of your return, we’ll need the following details:

| What we need | Why it matters |
|--------------|----------------|
| **Order number** (or receipt/email co

=== ANTI-TRAIT ===
I’m so sorry to hear you’re dealing with a family emergency—let’s get this return taken care of right away so you can focus on what matters most.

Here’s what we’ll do to fast‑track the process:

1. **Send me your order number**
